In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import weights_visualize as wv # weights visualization from RBM trained
from GBRBM import GBRBM # Gaussian-Binary RBM 
from sklearn import svm #SVM
from sklearn.neural_network import MLPClassifier #RNA-MLP
from sklearn import metrics
import matplotlib.pyplot as plt
np.random.seed(1)
path0 = os.getcwd()
# TODO: IMPORT SVM AND RNA FROM .py FILE, THEN REMOVE THE PRESENT CODE ON THIS NOTEBOOK!

In [2]:
def banco():
    # A 409, B 354, C 216 # TOTAL IMAGES AUGMENTED    
    os.chdir(path0)
    x_train = pd.read_csv("images_real.csv") # the file must be in notebook's folder
    x_train = np.array(x_train)[:,1:]
    
    for i in range(x_train.shape[1]):
        x_train[:,i] = (x_train[:,i] - np.mean(x_train[:,i])) / np.std(x_train[:,i]) # NORMALIZE to 0 mean and 1 std

    aux = np.ones((979))
    for z in range(979):
        if(z>=0 and z<=408):
            aux[z] = 0
        else:
            if(z>=409 and z<=763):
                aux[z] = 1
            else:
                aux[z] = 2
    aux = aux.astype(int)
    y_train = np.array(aux)
    y_train = aux.reshape((y_train.shape[0], 1))
    todos = np.concatenate((x_train, y_train), axis = 1)
    np.random.shuffle(todos) # shuffle data
    t = int(x_train.shape[0]*.2) # 20% of data for test
    x_train = todos[t:,:2538]
    y_train = todos[t:,2538]
    x_test = todos[:t,:2538]
    y_test = todos[:t,2538]
    return x_train, y_train, x_test, y_test

In [3]:
####################################
# LOAD DATA AND SET RBM PARAMETERS #
####################################
x_train, y_train, x_test, y_test = banco()

nvis = x_train.shape[1] # visible neurons (input layer) #
nhid = 60 # hidden neurons (latent ones) #
epoch = 200 # 200-500 epochs is good to convergence #
l_rate = 0.01 # learning rate #
k = 1 #sampling steps for alucinate data # 

In [4]:
def SVM(x_train, y_train, x_test, y_test):

    SVM = svm.SVC(C=100, gamma = 'scale')#, gamma=g, kernel=k) 
    SVM.fit(x_train, y_train) 
    previsao = SVM.predict(x_test) 
    acuracia= metrics.accuracy_score(y_test, previsao) 
    acuracia_b= metrics.balanced_accuracy_score(y_test, previsao)
    m_conf= metrics.confusion_matrix(y_test, previsao) 

    return acuracia, acuracia_b, m_conf

def RNAs(x_train, y_train, x_test, y_test, lri = float(0.01), lr = 'constant', hls=(100), mi=(400)):

    RNA= MLPClassifier(hidden_layer_sizes=hls, max_iter=mi, learning_rate=lr,learning_rate_init=lri) 
    RNA.fit(x_train, y_train) 
    previsao= RNA.predict(x_test)

    acuracia= metrics.accuracy_score(y_test, previsao) 
    acuracia_b= metrics.balanced_accuracy_score(y_test, previsao) 
    m_conf= metrics.confusion_matrix(y_test, previsao) 

    return acuracia, acuracia_b, m_conf

In [5]:
rep = 1 # number of repetitions
acc = np.zeros((rep))
bac = np.zeros((rep))
conf = np.zeros((rep))
acc2 = np.zeros((rep))
bac2 = np.zeros((rep))
for i in range(0,rep):
    np.random.seed(i)
    
    #x_train, y_train, x_test, y_test = banco()
    # GBRBM creation with adaptative temperature (True state) # 
    rbm = GBRBM(nhid, nvis, epoch, 20, l_rate , k, x_train, True, i) # Shift + Tab for see input parameters #
    w8, a, b = rbm.fit(x_train)
    teste = rbm.sigmoid(np.matmul(x_train, w8) + np.mean(rbm.b, axis = 0))    
    teste2 = rbm.sigmoid(np.matmul(x_test, w8) + np.mean(rbm.b, axis = 0))
    
    acc[i], bac[i], _ = SVM(teste, y_train, teste2, y_test)
    acc2[i], bac2[i], _ = RNAs(teste, y_train, teste2, y_test)
    
print("Acurácias Balanceadas-SVM", np.round(bac, 2))
print("Média Acurácias-SVM", np.round(np.mean(bac),2))

print("Acurácias Balanceadas-RNAs", np.round(bac2, 2))
print("Média Acurácias-RNAs", np.round(np.mean(bac2),2))

Epoch :0, Reconstruction Error :1664.0372
Epoch :100, Reconstruction Error :40.0842
Acurácias Balanceadas [0.77]
Média Acurácias 0.77
Acurácias Balanceadas [0.77]
Média Acurácias 0.77


In [6]:
"""""""""
SHOWS THE WEIGHT MATRIX FOR IMAGE PATTERN RECOGNITION
"""""""""
image = Image.fromarray(wv.tile_raster_images(X = w8.T,img_shape=(27, 94), tile_shape=(8,8),tile_spacing=(1,1)))
image = image.resize((500, 500))
image.show()